**获取研究期间剔除上市短、ST的标的**

In [2]:
import numpy as mp
import pandas as pd
import os
import jqdatasdk
import datetime

In [35]:
# jqdatasdk.auth("18867905508", "Ljy19950217") # user & password
jqdatasdk.auth('13350103318', '87654321wW')

In [34]:
jqdatasdk.get_query_count() # JQ总共能查多少data 还剩能查多少data

{'total': 1000000, 'spare': 962420}

In [21]:
start = '2016-12-31'
end = '2021-12-31'
dateList = pd.date_range(start, end, freq='m')

In [16]:
def delect_stop(stocks, beginDate, n=30*2):
    '''剔除上市不足60日的标的'''
    stockList = []
    beginDate = datetime.datetime.strptime(beginDate, '%Y-%m-%d')
    for stock in stocks:
        start_date = jqdatasdk.get_security_info(stock).start_date
        if start_date < (beginDate - datetime.timedelta(days=n)).date():
            stockList.append(stock)
    return stockList

def delect_st(stocks, beginDate):
    '''剔除st的标的'''
    st_data = jqdatasdk.get_extras('is_st', stocks, count=1, end_date=beginDate)
    stockList = [stock for stock in stocks if st_data[stock][0] == False]
    return stockList

In [27]:
hs300_origin_dict = {}
hs300_cleared_dict = {}
for date in dateList:
    '''获取交易标的'''
    date = date.strftime('%Y-%m-%d')
    hs300_list = jqdatasdk.get_index_stocks('000300.XSHG', date)
    hs300_origin_dict[date] = hs300_list
    hs300_cleared_dict[date] = delect_st(delect_stop(hs300_list, date), date)
# hs300_cleared_dict

In [28]:
import json
'''写入本地，方便之后调用'''
jsonObj = json.dumps(hs300_origin_dict)
fileObj = open('hs300_origin_dict.json', 'w', encoding='utf-8')
fileObj.write(jsonObj)
fileObj.close()

jsonObj = json.dumps(hs300_cleared_dict)
fileObj = open('hs300_cleared_dict.json', 'w', encoding='utf-8')
fileObj.write(jsonObj)
fileObj.close()

In [30]:
with open('hs300_cleared_dict.json', 'r') as f:
    symbol_dict = json.load(f)
symbol_dict.keys()